# 0. Data Loading and Exploration

## Mục tiêu

File này giúp bạn hiểu:
- **File GDF chứa gì?**
- **Dữ liệu EEG có cấu trúc như thế nào?**
- **Các thông tin quan trọng: channels, events, trials**

## 1. Load data và xem thông tin cơ bản

**Dataset:** BCI Competition IV - Dataset 2a  
**Subject:** A01 (Người thử nghiệm số 1)  
**Paradigm:** Motor Imagery (Tưởng tượng vận động)

In [1]:
import mne
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load file GDF
data_path = './data/'
raw = mne.io.read_raw_gdf(data_path + 'A01T.gdf', preload=True, verbose=False)

# Thông tin cơ bản
print("=" * 60)
print("THONG TIN FILE GDF")
print("=" * 60)
print(f"Subject: A01 (Nguoi thu nghiem 1)")
print(f"Sampling rate: {raw.info['sfreq']} Hz")
print(f"  -> Do 250 lan moi giay")
print(f"\nDuration: {raw.times[-1]:.1f} seconds ({raw.times[-1]/60:.1f} phut)")
print(f"  -> Tong thoi gian ghi EEG")
print(f"\nTotal samples: {len(raw.times):,}")
print(f"  -> Tong so lan do = {raw.times[-1]:.0f}s x 250Hz")
print(f"\nChannels: {len(raw.ch_names)}")
print(f"  -> 22 kenh EEG + 3 kenh EOG (dien cuc mat)")

THONG TIN FILE GDF
Subject: A01 (Nguoi thu nghiem 1)
Sampling rate: 250.0 Hz
  -> Do 250 lan moi giay

Duration: 2690.1 seconds (44.8 phut)
  -> Tong thoi gian ghi EEG

Total samples: 672,528
  -> Tong so lan do = 2690s x 250Hz

Channels: 25
  -> 22 kenh EEG + 3 kenh EOG (dien cuc mat)


## 2. Channel names - Vị trí các điện cực

**25 channels gồm:**
- **22 EEG channels:** Đặt trên đầu để đo tín hiệu não
- **3 EOG channels:** Đặt quanh mắt để đo chuyển động mắt (artifact)

**Vị trí quan trọng:**
- **C3:** Giữa đầu bên trái (motor cortex - điều khiển tay phải)
- **Cz:** Chính giữa đỉnh đầu
- **C4:** Giữa đầu bên phải (motor cortex - điều khiển tay trái)
- **Fz:** Giữa trán
- **Pz:** Giữa sau đầu

In [2]:
print("\n" + "=" * 60)
print("CHANNEL NAMES (Ten cac kenh)")
print("=" * 60)
print("\nTat ca 25 channels:")
for i, ch in enumerate(raw.ch_names, 1):
    print(f"{i:2d}. {ch}")

print("\n" + "-" * 60)
print("Channels quan trong nhat cho Motor Imagery:")
print("-" * 60)
motor_channels = ['EEG-C3', 'EEG-Cz', 'EEG-C4']
for ch in motor_channels:
    if ch in raw.ch_names:
        idx = raw.ch_names.index(ch)
        print(f"  {ch:10s} (index: {idx:2d}) - Motor cortex")


CHANNEL NAMES (Ten cac kenh)

Tat ca 25 channels:
 1. EEG-Fz
 2. EEG-0
 3. EEG-1
 4. EEG-2
 5. EEG-3
 6. EEG-4
 7. EEG-5
 8. EEG-C3
 9. EEG-6
10. EEG-Cz
11. EEG-7
12. EEG-C4
13. EEG-8
14. EEG-9
15. EEG-10
16. EEG-11
17. EEG-12
18. EEG-13
19. EEG-14
20. EEG-Pz
21. EEG-15
22. EEG-16
23. EOG-left
24. EOG-central
25. EOG-right

------------------------------------------------------------
Channels quan trong nhat cho Motor Imagery:
------------------------------------------------------------
  EEG-C3     (index:  7) - Motor cortex
  EEG-Cz     (index:  9) - Motor cortex
  EEG-C4     (index: 11) - Motor cortex


## 3. Events - Các sự kiện đánh dấu trials

**Event là gì?**
- Marker đánh dấu thời điểm xuất hiện mũi tên trên màn hình
- Người thử nghiệm thấy mũi tên → Bắt đầu tưởng tượng vận động

**Event format:** `[sample_index, duration, event_id]`

**Event IDs (Mã sự kiện):**
- **769:** Left hand (Tay trái)
- **770:** Right hand (Tay phải)
- **771:** Feet (Chân)
- **772:** Tongue (Lưỡi)
- **Các mã khác:** Markers kỹ thuật (bỏ qua)

In [3]:
# Trích xuất events
events, event_dict = mne.events_from_annotations(raw, verbose=False)

print("\n" + "=" * 60)
print("EVENTS (Cac su kien)")
print("=" * 60)
print(f"\nTong so events: {len(events)}")
print("\nVi du 5 events dau tien:")
print("[sample_index, duration, event_id]")
print(events[:5])

# Đếm từng loại event
print("\n" + "-" * 60)
print("Phan bo cac event IDs:")
print("-" * 60)
event_ids = events[:, 2]
unique_ids, counts = np.unique(event_ids, return_counts=True)

# Mapping event IDs sang ý nghĩa
event_meanings = {
    769: 'Left hand (Tay trai)',
    770: 'Right hand (Tay phai)',
    771: 'Feet (Chan)',
    772: 'Tongue (Luoi)'
}

for event_id, count in zip(unique_ids, counts):
    meaning = event_meanings.get(event_id, 'Marker ky thuat (bo qua)')
    print(f"Event ID {event_id}: {count:3d} lan - {meaning}")

# Đếm trials thực sự (4 classes chính)
main_events = [769, 770, 771, 772]
total_trials = sum(counts[unique_ids == eid][0] for eid in main_events if eid in unique_ids)
print(f"\nTong so trials (4 classes): {total_trials}")
print(f"  -> Moi class: khoang {total_trials // 4} trials")


EVENTS (Cac su kien)

Tong so events: 603

Vi du 5 events dau tien:
[sample_index, duration, event_id]
[[    0     0     5]
 [    0     0     3]
 [29683     0     5]
 [29683     0     4]
 [49955     0     5]]

------------------------------------------------------------
Phan bo cac event IDs:
------------------------------------------------------------
Event ID 1:  15 lan - Marker ky thuat (bo qua)
Event ID 2:   1 lan - Marker ky thuat (bo qua)
Event ID 3:   1 lan - Marker ky thuat (bo qua)
Event ID 4:   1 lan - Marker ky thuat (bo qua)
Event ID 5:   9 lan - Marker ky thuat (bo qua)
Event ID 6: 288 lan - Marker ky thuat (bo qua)
Event ID 7:  72 lan - Marker ky thuat (bo qua)
Event ID 8:  72 lan - Marker ky thuat (bo qua)
Event ID 9:  72 lan - Marker ky thuat (bo qua)
Event ID 10:  72 lan - Marker ky thuat (bo qua)

Tong so trials (4 classes): 0
  -> Moi class: khoang 0 trials


## 4. Cấu trúc dữ liệu EEG

**Dữ liệu EEG là một ma trận lớn:**
```
Shape: (n_channels, n_samples)
     = (25 channels, ~672,000 samples)
```

**Ý nghĩa:**
- **Mỗi hàng:** 1 channel (1 điện cực)
- **Mỗi cột:** 1 time point (1 thời điểm)
- **Mỗi giá trị:** Điện áp não tại thời điểm đó (đơn vị: microVolt)

**Ví dụ:**
- `data[0, 100]` = Giá trị điện áp của channel 1 tại sample thứ 100
- `data[7, :]` = Toàn bộ tín hiệu của channel C3 (index 7)

In [4]:
# Lấy dữ liệu signal
data = raw.get_data()

print("\n" + "=" * 60)
print("CAU TRUC DU LIEU EEG")
print("=" * 60)
print(f"\nShape: {data.shape}")
print(f"  -> ({data.shape[0]} channels, {data.shape[1]:,} samples)")

print(f"\nKich thuoc du lieu:")
print(f"  Total values: {data.shape[0] * data.shape[1]:,}")
print(f"  Memory: {data.nbytes / 1e6:.1f} MB")

print(f"\nVi du gia tri dien ap (5 sample dau tien cua channel C3):")
c3_idx = raw.ch_names.index('EEG-C3')
print(f"  Channel: EEG-C3 (index {c3_idx})")
print(f"  Values (microVolt): {data[c3_idx, :5]}")

print(f"\nThong ke channel C3:")
print(f"  Mean: {data[c3_idx].mean():.2f} µV")
print(f"  Std:  {data[c3_idx].std():.2f} µV")
print(f"  Min:  {data[c3_idx].min():.2f} µV")
print(f"  Max:  {data[c3_idx].max():.2f} µV")


CAU TRUC DU LIEU EEG

Shape: (25, 672528)
  -> (25 channels, 672,528 samples)

Kich thuoc du lieu:
  Total values: 16,813,200
  Memory: 134.5 MB

Vi du gia tri dien ap (5 sample dau tien cua channel C3):
  Channel: EEG-C3 (index 7)
  Values (microVolt): [-2.25097656e-05 -2.27539062e-05 -2.04101562e-05 -1.63574219e-05
 -1.83593750e-05]

Thong ke channel C3:
  Mean: -0.00 µV
  Std:  0.00 µV
  Min:  -0.00 µV
  Max:  0.00 µV
